In [1]:
import pandas as pd
import numpy as np
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('../data/processed/Cleaned_Dataframe.csv')

Remove columns that do not add value to classifying text

In [3]:
df = df.drop('Unnamed: 0', 1)
df = df.drop('Dataline', 1)
df = df.drop('PlayerLinenumber', 1)

Generate Cateogires to predict Player

In [4]:
df.Play = pd.Categorical(df.Play)
df.Player = pd.Categorical(df.Player)
df.PlayerLine = pd.Categorical(df.PlayerLine)

In [5]:
df['Play_code'] = df.Play.cat.codes
df['Player_code'] = df.Player.cat.codes
df['PlayerLine_code'] = df.PlayerLine.cat.codes

In [6]:
df = df.dropna()

In [7]:
features = ['Play_code','PlayerLine_code', 'Act', 'Scene', 'Line']
x = np.array(df[features])
y = np.array(df['Player_code'])

Split data for training and testing, using 30% of the data for testing. Set the random_sate to 1 so the tests can be rerun for same results.

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=1)

In [ ]:
model_lg = LogisticRegression()
model_lg.fit(x_train, y_train)
y_pred = model_lg.predict(x_test)

In [ ]:
model_rf = RandomForestClassifier()
model_rf.fit(x_train, y_train)
y_pred = model_rf.predict(x_test)
print(metrics.classification_report(y_test, y_pred))